# Train the Neural Mesh Simplification model

## Set up the environment

In [2]:
!pip install -r requirements.txt

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html, https://data.pyg.org/whl/torch-2.4.0+cpu.html, https://data.pyg.org/whl/torch-2.4.0+cpu.html


In [3]:
!pip install -e .

Obtaining file:///Users/gennaro/code/thinAir/ml/neural-mesh-simplification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for neural-mesh-simplification (pyproject.toml) ... done
  Created wheel for neural-mesh-simplification: filename=neural_mesh_simplification-0.1.0-0.editable-py3-none-any.whl size=5136 sha256=28d9cf0da394a9fcd19db66c5a703a4340c026242fd6fc09a6416d87789d077e
  Stored in directory: /private/var/folders/nh/2tqsxc9x06q9nk4n_bm862hr0000gn/T/pip-ephem-wheel-cache-7jdnc4y9/wheels/52/59/e0/2e73754b23001707e03f3788c7516d3d274ab5357b62a28d2a
Successfully built neural-mesh-simplification
  Attempting uninstall: neural-mesh-simplification
    Found existing installation: neural-mesh-simplification 0.1.0
    Uninstalling neural-mesh-simplification-0.1.0:
      Successfully uninstalled neural-mesh-simpl

## Load the default config

In [4]:
def load_config(config_path):
    import yaml
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
    return config

config = load_config("configs/default.yaml")

## Set up the logging level

In [5]:
import logging
logging.basicConfig(level=logging.INFO)

## Start the training

In [ ]:
from neural_mesh_simplification.trainer.trainer import Trainer

trainer = Trainer(config)

try:
    trainer.train()
except Exception as e:
    trainer.handle_error(e)
    trainer.save_training_state(os.path.join(config["training"]["checkpoint_dir"], "training_state.pth"))

INFO:neural_mesh_simplification.trainer.trainer:Initializing trainer...
INFO:neural_mesh_simplification.trainer.trainer:Using device: cpu
INFO:neural_mesh_simplification.trainer.trainer:Initializing model...
INFO:neural_mesh_simplification.trainer.trainer:Setting up optimizer and loss...
/opt/homebrew/Caskroom/miniconda/base/envs/neural-mesh-simplification/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
INFO:neural_mesh_simplification.trainer.trainer:Preparing data loaders...
INFO:neural_mesh_simplification.trainer.trainer:Loading dataset from data/processed
INFO:neural_mesh_simplification.trainer.trainer:Dataset size: 4
INFO:neural_mesh_simplification.trainer.trainer:Splitting dataset: 3 train, 1 validation
INFO:neural_mesh_simplification.trainer.trainer:Data loaders prepared successfully
INFO:neural_mesh_simplification.trainer.trainer:Trainer initializ

Initialized ProbabilisticSurfaceDistanceLoss with k=3, num_samples=100


/opt/homebrew/Caskroom/miniconda/base/envs/neural-mesh-simplification/lib/python3.12/site-packages/torch_sparse/matmul.py:97: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:55.)
  C = torch.sparse.mm(A, B)
/Users/gennaro/code/thinAir/ml/neural-mesh-simplification/src/neural_mesh_simplification/losses/overlapping_triangles_loss.py:159: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Cross.cpp:66.)
  cross_prod = torch.cross(v1 - v0, v2 - v0)
INFO:root:Epoch [1/2